In [ ]:
%pip install --upgrade pip 
%pip install sqlalchemy
%pip install sqlalchemy_utils
%pip install psycopg2
%pip install faker

In [ ]:
from enum import Enum
from typing import List
from uuid import UUID, uuid4
from datetime import datetime
from contextlib import closing
from time import time
from random import choice as random_choice

from sqlalchemy import (
    create_engine,
    inspect,
    ForeignKey,
    UniqueConstraint,
    types
)
from sqlalchemy.orm import Mapped, mapped_column, relationship, validates, declarative_base, sessionmaker
from sqlalchemy_utils import database_exists, create_database

from faker import Faker

In [ ]:
Base = declarative_base()



class LikeDislike(Enum):
    like = 1
    dislike = -1


class Review(Base):
    __tablename__ = "review"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_review_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID]
    movie_id: Mapped[UUID] = mapped_column(index=True)

    score: Mapped[int] = mapped_column(nullable=False, comment="Оценка отзыва")
    text: Mapped[str] = mapped_column(nullable=False, comment="Текст отзыва")
    review_dt: Mapped[datetime] = mapped_column(nullable=False)

    reviews_ratings: Mapped[List['UserReviewRating']] = relationship(
        back_populates="review", cascade='all, delete')
    
    @validates('score')
    def validate_score(self, key, value):
        if 0 <= value <= 10:
            return value
        raise ValueError("Value should be between 0 and 10")



class Bookmark(Base):
    __tablename__ = "bookmark"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_bookmark_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID] = mapped_column(index=True)
    movie_id: Mapped[UUID] = mapped_column(types.Uuid)


class UserReviewRating(Base):
    __tablename__ = "user_review_rating"
    __table_args__ = (
        UniqueConstraint('user_id', 'review_id', name='unique_user_review_rating'),
    )
    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID] = mapped_column(index=True)

    review: Mapped["Review"] = relationship(back_populates="reviews_ratings")
    review_id: Mapped[UUID] = mapped_column(ForeignKey("review.id"))

    score: Mapped[LikeDislike]


In [ ]:

user = 'bench_user'
password = 'bench_pass'
host = 'localhost'
port = 5434
db = 'bench_db'

pg_url = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
if not database_exists(pg_url):
    print('No db')
    create_database(pg_url)
    print('db created')

engine = create_engine(pg_url)

inspector = inspect(engine)
db_tables = inspector.get_table_names()

if not db_tables:
    print('Моделей нет в БД')
    Base.metadata.create_all(bind=engine)
    print('Модели добавлены в БД')
else:
    print('Моделей уже есть в БД')

In [ ]:
fake: Faker = Faker()

USERS_COUNT = 2000
USERS_IDS = [uuid4() for _ in range(USERS_COUNT)]

MOVIES_COUNT = 500
MOVIES_IDS = [uuid4() for _ in range(MOVIES_COUNT)]

REVIEWS_COUNT = USERS_COUNT * MOVIES_COUNT

session_factory = sessionmaker(bind=engine, expire_on_commit=False)

In [ ]:
# insert review
with closing(session_factory()) as session:
    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_review = Review(
                user_id=user_id,
                movie_id=movie_id,
                score=fake.random_int(min=0, max=10),
                text=fake.text(), # fake.paragraph,
                review_dt=fake.date_time_between(start_date="-1y", end_date="now")
            )

            session.add(new_review)

    session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: float = round(REVIEWS_COUNT / insertion_time, 2)

    print('Insertion speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# insert bookmarks
with closing(session_factory()) as session:
    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_bookmark = Bookmark(
                user_id=user_id,
                movie_id=movie_id
            )

            session.add(new_bookmark)

    session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: float = round(REVIEWS_COUNT / insertion_time, 2)

    print('Insertion speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# insert review ratings
with closing(session_factory()) as session:
    query_result = session.query(Review.id).limit(2000).all()
    all_reviews_ids = [review.id for review in query_result]
    start_time: float = time()
    for user_id in USERS_IDS[:500]:
        for review_id in all_reviews_ids:
            score = random_choice([LikeDislike.like, LikeDislike.dislike])
            new_review_rating = UserReviewRating(
                user_id=user_id,
                review_id=review_id,
                score=score
            )

            session.add(new_review_rating)

    session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: float = round(REVIEWS_COUNT / insertion_time, 2)

    print('Insertion speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# select reviews
with closing(session_factory()) as session:
    start_time: float = time()
    all_reviews = session.query(Review).all()

    insertion_time: float = time() - start_time
    insertion_speed: float = round(REVIEWS_COUNT / insertion_time , 2)

    print('Selecting speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# select bookmarks
with closing(session_factory()) as session:
    start_time: float = time()
    all_reviews = session.query(Bookmark).all()

    insertion_time: float = time() - start_time
    insertion_speed: float = round(REVIEWS_COUNT / insertion_time , 2)

    print('Selecting speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# select review ratings
with closing(session_factory()) as session:
    start_time: float = time()
    all_reviews = session.query(UserReviewRating).all()

    insertion_time: float = time() - start_time

    reviews_ratings_count = len(session.query(UserReviewRating).all())
    insertion_speed: float = round(reviews_ratings_count / insertion_time , 2)

    print('Selecting speed: {:,} records/sec'.format(insertion_speed))

In [ ]:
# update review
with closing(session_factory()) as session:
    start_time: float = time()
    users_count_for_tests = USERS_COUNT / 20
    for user_id in USERS_IDS[:users_count_for_tests]:
        for movie_id in MOVIES_IDS:
            review_to_update = session.query(Review).filter(Review.user_id==user_id,
                                                            Review.movie_id==movie_id).one_or_none()
            
            review_to_update.score=fake.random_int(min=0, max=10)
            review_to_update.text=fake.paragraph(), # fake.paragraph,
            review_dt=fake.date_time_between(start_date="-1y", end_date="now")

    session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: float = round((REVIEWS_COUNT/ 20) / insertion_time, 2)

    print('Update speed: {:,} records/sec'.format(insertion_speed))